In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
tips = sns.load_dataset('tips')

## 최적화 Tip 정리

### 0. np.array_equal - 코드 변화 결과 확인

최적화한 결과가 원래 사용하던 코드와 일치하는 지 간단하게 확인할 수 있다.

In [8]:
# 1번 예제 확인
np.array_equal((tips['tip'] > 5) & (tips['sex'] =='Male'), (tips['tip'].values > 5) & (tips['sex'].values =='Male'))

True

참고 : pandas에도 방법이 존재 

단, 비교 타입이 series든 ndarray든 타입이 일치해야만 한다.

In [ ]:
import pandas.util.testing as tm
tm.assert_equal(pd.Series([1,2,3]).values, pd.Series([4,5,6]).values)

### 1. 조건식 컬럼에 .values를 붙이기

.values가 ndarray를 호출 // series보다 ndarray 연산이 더 빠르다.

In [12]:
%timeit tips.loc[tips['tip'] > 5 & (tips['sex'] =='Male')]

1.47 ms ± 9.54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
# .values만 붙인 후 속도 개선
%timeit tips.loc[(tips['tip'].values > 5) & (tips['sex'].values =='Male') ]

839 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 2. iterrows 대신 itertuples

참고 : https://medium.com/swlh/why-pandas-itertuples-is-faster-than-iterrows-and-how-to-make-it-even-faster-bc50c0edd30d

namedtuple을 활용하기 때문에 iterrows보다 훨씬 빠르다고 한다. 생각보다 큰 효과를 볼 수 있다.

[itertuples 사용 설명](#itertuples)

### 3. apply 함수 대신 pandas built-in 함수 활용

python으로 함수를 작성하는 것보다 built-in 함수를 최대한 조합하는 것이 속도가 훨씬 빠르다.

### 4. np.vectorize 혹은 numpy built-in 함수 활용

numpy built-in 함수가 더 빠르다. // 함수의 input은 .values 즉, ndarray형태로 만든다.

### 4. 

### single value에는 .loc (.iloc) 대신 .at(.iat)

### itertuples

In [17]:
import pandas as pd
import seaborn as sns
tips = sns.load_dataset('tips').head(3)

In [28]:
for row in tips.itertuples():
    print(row)

Pandas(Index=0, total_bill=16.99, tip=1.01, sex='Female', smoker='No', day='Sun', time='Dinner', size=2)
Pandas(Index=1, total_bill=10.34, tip=1.66, sex='Male', smoker='No', day='Sun', time='Dinner', size=3)
Pandas(Index=2, total_bill=21.01, tip=3.5, sex='Male', smoker='No', day='Sun', time='Dinner', size=3)


In [30]:
# .으로 컬럼 접근
for row in tips.itertuples(index=False):
    print(row.total_bill, row.tip)

16.99 1.01
10.34 1.66
21.01 3.5


In [36]:
# 특정 컬럼을 지정하면 컬럼마다 값을 분리
for index, day, time in tips[['day','time']].itertuples():
    print(index, day, time)

0 Sun Dinner
1 Sun Dinner
2 Sun Dinner
